In [3]:
from collections import defaultdict, Counter
import glob
import os

from IPython.core.display import HTML, Image
from matplotlib_venn import venn3
import pandas as pd
import pybedtools
import pysam

from gscripts.general import parsers
%load_ext autoreload
%autoreload 2
reload(parsers)
reload(pybedtools)

<module 'pybedtools' from '/nas3/gpratt/anaconda/lib/python2.7/site-packages/pybedtools-0.6.8-py2.7-linux-x86_64.egg/pybedtools/__init__.pyc'>

In [13]:
!rm -rf /nas3/gpratt/projects/encode/analysis/encode_v2/
!mkdir -p /nas3/gpratt/projects/encode/analysis/encode_v2/
!scp -q tscc-login.sdsc.edu:~/projects/encode/analysis/encode_v2/*.{metrics,final.out,peaks.bed} /nas3/gpratt/projects/encode/analysis/encode_v2/

In [14]:
analysis_dir = "/nas3/gpratt/projects/encode/analysis/encode_v2/"
encode_v10 = parsers.clipseq_metrics(analysis_dir, iclip=True)

encode_v10["Reads After Triming"] = encode_v10['Input Reads'] - encode_v10['Too short reads']
encode_v10["Fraction Collapsed"] = encode_v10['Usable Reads'] / encode_v10['Uniquely Mapped Reads'].astype(float)
encode_v10["Fraction Usable"] = encode_v10['Usable Reads'] / encode_v10['Input Reads'].astype(float)

filtered_encode_v10 = encode_v10[[ "Input Reads",
                            "Reads After Triming",
                            "repetitive_count",
                            "Uniquely Mapped Reads",
                            "Uniquely mapped reads %",
                            "Usable Reads",
                            "Fraction Collapsed",
                            "Fraction Usable",
                            #"spot",
                            "Num Peaks",
                            "Passed QC"
                            ]]

HTML(filtered_encode_v10.to_html(formatters={"Input Reads" : parsers.commas,
                                     "Reads After Triming" : parsers.commas,
                                     "Reads after Quality Filtering" : parsers.commas,
                                     "Uniquely Mapped Reads" : parsers.commas,
                                     "Usable Reads" : parsers.commas,
                                     "Num Peaks": parsers.commas
                                     } ))

<closed file '/nas3/gpratt/projects/encode/analysis/encode_v2/2F-input_S10_R2.unassigned.polyATrim.adapterTrim.metrics', mode 'r' at 0xe0356f0>


,Input Reads,Reads After Triming,repetitive_count,Uniquely Mapped Reads,Uniquely mapped reads %,Usable Reads,Fraction Collapsed,Fraction Usable,Num Peaks,Passed QC
10M_S18_R2.unassigned,"15,554,259","15,277,165",3797389,"6,225,471",54.57%,"3,383,515",0.543495,0.217530,"83,674",True
10S_S19_R2.unassigned,"10,796,032","10,349,118",1276203,"4,715,419",52.64%,"2,890,184",0.612922,0.267708,"4,178",True
10_GM12878_CLIP_1_Infminus_AND_12_GM12878_CLIP_1_Infplus_ATTCAGAA-ATAGAGGC_R2.A03_10_GM12878_CLIP_1_Infminus_R60,"20,771,980","17,081,469",8023505,"2,498,518",27.58%,"522,334",0.209058,0.025146,"9,811",True
10_GM12878_CLIP_1_Infminus_AND_12_GM12878_CLIP_1_Infplus_ATTCAGAA-ATAGAGGC_R2.A04_12_GM12878_CLIP_1_Infplus_R60,"9,017,092","7,813,205",3458297,"1,235,046",28.36%,"284,792",0.230592,0.031584,"4,372",False
10_GM12878_CLIP_1_Infminus_AND_12_GM12878_CLIP_1_Infplus_ATTCAGAA-ATAGAGGC_R2.F05_12_GM12878_CLIP_1_Infplus_R60,"9,247,607","7,651,262",3453856,"1,243,853",29.63%,"320,619",0.257763,0.034670,"5,146",False
10_GM12878_CLIP_1_Infminus_AND_12_GM12878_CLIP_1_Infplus_ATTCAGAA-ATAGAGGC_R2.G07_10_GM12878_CLIP_1_Infminus_R60,"5,976,139","5,146,684",2217738,"1,063,244",36.30%,"159,695",0.150196,0.026722,"2,374",False
10_GM12878_CLIP_1_Infminus_R60merged.peaks,0,0,NaN,0,NaN,0,NaN,NaN,"13,168",False
11_GM12878_Input_1_Infplus_GAGATTCC-ATAGAGGC_R2.A04,"108,180","105,202",101833,"1,303",38.68%,"1,022",0.784344,0.009447,11,False
11_GM12878_Input_1_Infplus_GAGATTCC-ATAGAGGC_R2.unassigned,"34,515,438","33,545,910",13443909,"6,250,268",31.61%,"3,745,438",0.599244,0.108515,"38,515",True
12_GM12878_CLIP_1_Infplus_R60merged.peaks,0,0,NaN,0,NaN,0,NaN,NaN,"10,841",False


In [15]:
filtered_encode_v10[["Inf" in item for item in filtered_encode_v10.index]]

,Input Reads,Reads After Triming,repetitive_count,Uniquely Mapped Reads,Uniquely mapped reads %,Usable Reads,Fraction Collapsed,Fraction Usable,Num Peaks,Passed QC
10_GM12878_CLIP_1_Infminus_AND_12_GM12878_CLIP_1_Infplus_ATTCAGAA-ATAGAGGC_R2.A03_10_GM12878_CLIP_1_Infminus_R60,20771980,17081469,8023505,2498518,27.58%,522334,0.209058,0.025146,9811,True
10_GM12878_CLIP_1_Infminus_AND_12_GM12878_CLIP_1_Infplus_ATTCAGAA-ATAGAGGC_R2.A04_12_GM12878_CLIP_1_Infplus_R60,9017092,7813205,3458297,1235046,28.36%,284792,0.230592,0.031584,4372,False
10_GM12878_CLIP_1_Infminus_AND_12_GM12878_CLIP_1_Infplus_ATTCAGAA-ATAGAGGC_R2.F05_12_GM12878_CLIP_1_Infplus_R60,9247607,7651262,3453856,1243853,29.63%,320619,0.257763,0.034670,5146,False
10_GM12878_CLIP_1_Infminus_AND_12_GM12878_CLIP_1_Infplus_ATTCAGAA-ATAGAGGC_R2.G07_10_GM12878_CLIP_1_Infminus_R60,5976139,5146684,2217738,1063244,36.30%,159695,0.150196,0.026722,2374,False
10_GM12878_CLIP_1_Infminus_R60merged.peaks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13168,False
11_GM12878_Input_1_Infplus_GAGATTCC-ATAGAGGC_R2.A04,108180,105202,101833,1303,38.68%,1022,0.784344,0.009447,11,False
11_GM12878_Input_1_Infplus_GAGATTCC-ATAGAGGC_R2.unassigned,34515438,33545910,13443909,6250268,31.61%,3745438,0.599244,0.108515,38515,True
12_GM12878_CLIP_1_Infplus_R60merged.peaks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10841,False
13_GM12878_Input_2_Infminus_ATTACTCG-ATAGAGGC_R2.A04,110698,107388,104348,1107,36.41%,922,0.832882,0.008329,6,False
13_GM12878_Input_2_Infminus_ATTACTCG-ATAGAGGC_R2.unassigned,29196970,28504845,12136213,4702180,28.78%,2995738,0.637096,0.102604,32149,True


In [16]:
filtered_encode_v10[["A2B1" in item for item in filtered_encode_v10.index]]

,Input Reads,Reads After Triming,repetitive_count,Uniquely Mapped Reads,Uniquely mapped reads %,Usable Reads,Fraction Collapsed,Fraction Usable,Num Peaks,Passed QC
F1_IN_A2B1_ATTACTCG-TATAGCCT_L008_R2.A04,45039,42963,41378,788,49.72%,767,0.973350,0.017030,11,False
F1_IN_A2B1_ATTACTCG-TATAGCCT_L008_R2.unassigned,15194880,14438838,7277122,3508620,48.99%,3053848,0.870384,0.200979,33618,True
F1_IP_A2B1_TCCGGAGA-TATAGCCT_L008_R2.A01_F1_IP_mnCLIP_A2B1_high_1_hnRNPA2B1,5309703,3679762,469168,2395351,74.61%,2263281,0.944864,0.426254,12908,True
F1_IP_A2B1_TCCGGAGA-TATAGCCT_L008_R2.B06_F1_IP_mnCLIP_A2B1_high_1_hnRNPA2B1,2243962,1692626,244189,1108280,76.52%,1078014,0.972691,0.480407,5484,True
F1_IP_mnCLIP_A2B1_high_1_hnRNPA2B1merged.peaks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18442,False
F2_IN_A2B1_CGCTCATT-TATAGCCT_L008_R2.unassigned,12645375,11979392,5929899,3011711,49.78%,2675736,0.888444,0.211598,28408,True
F2_IP_A2B1_GAGATTCC-TATAGCCT_L008_R2.A01,50796,34679,5529,20848,71.52%,18884,0.905794,0.371762,43,False
F2_IP_A2B1_GAGATTCC-TATAGCCT_L008_R2.C01_F2_IP_mnCLIP_A2B1_high_2_hnRNPA2B1,5881739,4520826,666599,2817518,73.10%,2500503,0.887484,0.425130,19232,True
F2_IP_A2B1_GAGATTCC-TATAGCCT_L008_R2.D08fixed_F2_IP_mnCLIP_A2B1_high_2_hnRNPA2B1,2537590,2103793,352673,1312562,74.96%,1199078,0.913540,0.472526,8660,True
F2_IP_mnCLIP_A2B1_high_2_hnRNPA2B1merged.peaks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26222,False


In [17]:
filtered_encode_v10[["KK_" in item for item in filtered_encode_v10.index]]

,Input Reads,Reads After Triming,repetitive_count,Uniquely Mapped Reads,Uniquely mapped reads %,Usable Reads,Fraction Collapsed,Fraction Usable,Num Peaks,Passed QC
KK_10_CLIPin_Taf_GAGATTCC-GGCTCTGA_R2.unassigned,26551140,25563030,8602524,10399239,62.21%,9218069,0.886418,0.347182,43568,True
KK_11_CLIPip_Taf_ATTCAGAA-GGCTCTGA_R2.A01_KK_11_mnCLIP_Taf15_IP_high_1_TAF15,8604499,7794388,614166,5527895,76.99%,4527433,0.819016,0.526170,28524,True
KK_11_CLIPip_Taf_ATTCAGAA-GGCTCTGA_R2.B06_KK_11_mnCLIP_Taf15_IP_high_1_TAF15,14295397,12992909,1048462,9255492,77.49%,6666709,0.720298,0.466354,39930,True
KK_11_CLIPip_Taf_ATTCAGAA-GGCTCTGA_R2.C01,470063,423093,33538,299759,76.95%,268960,0.897254,0.572179,922,False
KK_11_CLIPip_Taf_ATTCAGAA-GGCTCTGA_R2.D08fixed,276148,245691,19757,174858,77.39%,152870,0.874252,0.553580,371,False
KK_11_mnCLIP_Taf15_IP_high_1_TAF15merged.peaks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66285,False
KK_12_CLIPip_Taf_GAATTCGT-GGCTCTGA_R2.A01_KK_12__mnCLIP_Taf15_IP_low_1_TAF15,26486,23737,1926,16932,77.63%,16731,0.988129,0.631692,26,False
KK_12_CLIPip_Taf_GAATTCGT-GGCTCTGA_R2.B06_KK_12__mnCLIP_Taf15_IP_low_1_TAF15,57776,50301,4051,35565,76.90%,35088,0.986588,0.607311,53,False
KK_12_CLIPip_Taf_GAATTCGT-GGCTCTGA_R2.C01,19346393,17585113,1315443,12680566,77.94%,10420758,0.821790,0.538641,44810,True
KK_12_CLIPip_Taf_GAATTCGT-GGCTCTGA_R2.D08fixed,10622189,9661952,727137,6984695,78.17%,6132611,0.878007,0.577340,34224,True


In [18]:
filtered_encode_v10[["Stefan_" in item for item in filtered_encode_v10.index]]

,Input Reads,Reads After Triming,repetitive_count,Uniquely Mapped Reads,Uniquely mapped reads %,Usable Reads,Fraction Collapsed,Fraction Usable,Num Peaks,Passed QC
Stefan-1to200-Imp1_S13_R2.A01_Stefan_1to200_Imp1_IGF2BP1,9071747,8713722,1758866,5226015,75.14%,4837188,0.925598,0.533215,88023,True
Stefan-1to200-Imp1_S13_R2.B06_Stefan_1to200_Imp1_IGF2BP1,5958486,5776283,1162443,3470402,75.22%,3266662,0.941292,0.548237,67339,True
Stefan-1to200-Imp2_S14_R2.C01_Stefan_1to200_Imp2_IGF2BP2,7697506,7536695,2116414,4018517,74.14%,2679996,0.666912,0.348164,60510,True
Stefan-1to200-Imp2_S14_R2.D08fixed_Stefan_1to200_Imp2_IGF2BP2,7182062,7093170,1975010,3819917,74.63%,2773978,0.726188,0.386237,60906,True
Stefan-1to25-Imp1_S16_R2.A01_Stefan_1to25_Imp1_IGF2BP1,2779851,2757690,353877,1951841,81.20%,1867308,0.956691,0.671730,42712,True
Stefan-1to25-Imp1_S16_R2.B06_Stefan_1to25_Imp1_IGF2BP1,2006693,1994234,246992,1424165,81.51%,1375089,0.965541,0.685251,32182,True
Stefan-1to25-Imp2_S17_R2.C01_Stefan_1to25_Imp2_IGF2BP2,2985502,2870211,635006,1685864,75.42%,1331535,0.789823,0.446000,33357,True
Stefan-1to25-Imp2_S17_R2.D08fixed_Stefan_1to25_Imp2_IGF2BP2,2668509,2605733,548774,1558351,75.76%,1293459,0.830018,0.484712,31627,True
Stefan_1to200_Imp1_IGF2BP1merged.peaks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,112328,False
Stefan_1to200_Imp2_IGF2BP2merged.peaks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90358,False
